In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
import pandas as pd
import numpy as np
import os
import sys
from PIL import Image
sys.modules['Image'] = Image

labels_df = pd.read_csv('./spectrograms/labels.csv')

def isValidImage(imagePath, targetSize=(100, 100)):
    image = Image.open(imagePath)
    return image.size == targetSize

def loadAndPreprocessImage(filePath, targetSize=(100, 100)):
    image = Image.open(filePath)
    image = np.array(image) / 255.0  # normalize the image
    return image

class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, directory, xCol, yCol, targetSize=(100, 100), batchSize=32, classMode='categorical', subset=None):
        self.dataframe = dataframe
        self.directory = directory
        self.xCol = xCol
        self.yCol = yCol
        self.targetSize = targetSize
        self.batchSize = batchSize
        self.classMode = classMode
        self.subset = subset
        self.datagen = ImageDataGenerator(validation_split=0.2)
        self.validDataframe = self._filterDataframe()
        self.samples = len(self.validDataframe)
        self.generator = self.datagen.flow_from_dataframe(
            dataframe=self.validDataframe,
            directory=self.directory,
            x_col=self.xCol,
            y_col=self.yCol,
            target_size=self.targetSize,
            batch_size=self.batchSize,
            class_mode=self.classMode,
            subset=self.subset
        )

    def _filterDataframe(self):
        validRows = []
        for _, row in self.dataframe.iterrows():
            imagePath = os.path.join(self.directory, row[self.xCol])
            if isValidImage(imagePath, self.targetSize):
                validRows.append(row)
        return pd.DataFrame(validRows)

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        return self.generator[index]

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation data

# Assuming trainGenerator and validationGenerator are defined elsewhere with the paths to their respective data
trainGenerator = train_datagen.flow_from_directory(
    'path_to_train_data',
    target_size=(150, 150),  # Adjust based on your dataset
    batch_size=32,
    class_mode='categorical'  # Change to 'categorical' for multi-class classification
)

validationGenerator = validation_datagen.flow_from_directory(
    'path_to_validation_data',
    target_size=(150, 150),  # Adjust based on your dataset
    batch_size=32,
    class_mode='categorical'  # Change to 'categorical' for multi-class classification
)

# Model Architecture with Transfer Learning
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Regularization
    layers.Dense(3, activation='softmax')  # Change the number of units to match the number of classes
])

model.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',  # Change loss function for multi-class classification
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

# Train model
history = model.fit(
    trainGenerator,
    steps_per_epoch=trainGenerator.samples // trainGenerator.batch_size,
    validation_data=validationGenerator,
    validation_steps=validationGenerator.samples // validationGenerator.batch_size,
    epochs=50,  # Increased epochs with early stopping
    callbacks=[early_stopping, reduce_lr]
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_train_data'